# Geometry optimization

We use DFTK and the [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
package to find the minimal-energy bond length of the $H_2$ molecule.
First we set up an appropriate `DFTKCalculator` (see AtomsCalculators integration),
for which we use the LDA model just like in the Tutorial for silicon
in combination with a pseudodojo pseudopotential (see Pseudopotentials).

In [1]:
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
calc = DFTKCalculator(;
    model_kwargs = (; functionals=LDA(), pseudopotentials),  # model_DFT keyword arguments
    basis_kwargs = (; kgrid=[1, 1, 1], Ecut=10)  # PlaneWaveBasis keyword arguments
)

DFTKCalculator(functionals=Xc(lda_x, lda_c_pw), pseudopotentials=PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf"), Ecut=10, kgrid=[1, 1, 1])

Next we set up an initial hydrogen molecule within a box of vacuum.
We use the parameters of the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/)
and DFTK's AtomsBase integration to setup the hydrogen molecule.
We employ a simulation box of 10 bohr times 10 bohr times 10 bohr and a
pseudodojo pseudopotential.

In [2]:
using LinearAlgebra
using Unitful
using UnitfulAtomic

r0 = 1.4   # Initial bond length in Bohr
a  = 10.0  # Box size in Bohr

cell_vectors = [[a, 0, 0]u"bohr", [0, a, 0]u"bohr", [0, 0, a]u"bohr"]
h2_crude = periodic_system([:H => [0, 0, 0.]u"bohr",
                            :H => [r0, 0, 0]u"bohr"],
                           cell_vectors)

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [       0,        0,        0]u"a₀")
    Atom(H,  [     1.4,        0,        0]u"a₀")


Finally we call `minimize_energy!` to start the geometry optimisation.
We use `verbosity=2` to get some insight into the minimisation.
With `verbosity=1` only a summarising table would be printed and with
`verbosity=0` (default) the minimisation would be quiet.

In [3]:
using GeometryOptimization
results = minimize_energy!(h2_crude, calc; tol_forces=2e-6, verbosity=2)
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -1.110282381972                   -0.82   0.80    8.0    4.08s
  2   -1.117191702987       -2.16       -1.82   0.80    1.0    2.73s
  3   -1.117250264002       -4.23       -2.66   0.80    1.0   19.3ms
  4   -1.117251216249       -6.02       -3.30   0.80    1.0   21.9ms
  5   -1.117251300360       -7.08       -3.77   0.80    1.0   33.4ms
  6   -1.117251310001       -8.02       -4.86   0.80    1.0   19.3ms
  7   -1.117251310375       -9.43       -5.14   0.80    2.0   23.8ms
  8   -1.117251310379      -11.34       -5.59   0.80    1.0   20.4ms
  9   -1.117251310381      -11.80       -6.54   0.80    1.0   23.3ms
 10   -1.117251310381      -13.30       -7.26   0.80    2.0   25.6ms
 11   -1.117251310381      -15.18       -7.62   0.80    1.0   21.8ms
 12   -1.117251310381   +  -15.05       -7.90   0.80    1.0   22.2ms
 13   -1.117251310381      -14.35  

Structure after optimisation (note that the atom has wrapped around)

In [4]:
results.system

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [-0.0431828, -7.14517e-11, -3.1471e-10]u"a₀")
    Atom(H,  [ 1.44318, -6.98109e-11, -3.10227e-10]u"a₀")


Compute final bond length:

In [5]:
rmin = norm(position(results.system[1]) - position(results.system[2]))
println("Optimal bond length: ", rmin)

Optimal bond length: 1.4863655847396426 a₀


Our results (1.486 Bohr) agrees with the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/).